In [1]:
orbits = pd.read_csv('GMagAO-X-RV-targets-with-orbital-params.csv')
orbits

,pl_orbsmax,pl_orbper,pl_orbincl,pl_orbeccen,pl_masse,pl_msinie,st_mass,sy_dist,Re,pl_orblper,...,pl_mass_or_msini_e,contrast_at_highest_contrast,separation_at_highest_contrast,phase_at_highest_contrast,separation_at_largest_separation,contrast_at_largest_separation,phase_at_largest_separation,separation_at_average_separation,contrast_at_average_separation,phase_at_average_separation
0,2.8000,1796.4000,NaN,0.338,NaN,1506.45000,0.90,17.9323,11.981839,22.580,...,1506.45000,0.000014,0.508338,30.033153,32.457259,1.017854e-09,96.993692,26.322074,2.903092e-09,70.559375
1,1.6600,798.5000,NaN,0.680,NaN,565.73740,1.08,21.1397,11.686000,NaN,...,565.73740,0.000045,0.234243,30.061647,21.014353,1.952731e-09,91.511273,15.417389,6.259833e-09,68.756457
2,0.9950,335.1000,NaN,0.289,NaN,3140.03000,1.16,44.6570,11.663215,307.000,...,3140.03000,0.000275,0.044953,30.005568,4.074614,1.638186e-08,77.780903,3.701471,2.823662e-08,60.253707
3,2.1000,1076.6000,NaN,0.012,NaN,778.68350,1.06,13.7967,11.843042,147.000,...,778.68350,0.000019,0.557301,30.026513,24.511418,3.675762e-09,89.328610,24.274389,4.401018e-09,83.153894
4,3.6000,2391.0000,NaN,0.098,NaN,171.62100,1.06,13.7967,10.502257,295.000,...,171.62100,0.000016,0.551462,30.007385,43.164092,1.009152e-09,86.328864,41.810055,1.383826e-09,75.027425
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
405,0.2300,39.6450,NaN,0.028,NaN,349.61300,1.00,17.4671,11.337947,30.000,...,349.61300,0.002863,0.034617,30.013875,2.150518,2.637201e-07,90.598086,2.100633,3.444084e-07,81.580449
406,0.4157,102.5570,NaN,0.044,NaN,28.12781,0.91,17.4671,5.924378,16.000,...,28.12781,0.017942,0.007226,30.000185,3.955365,2.113485e-08,90.853192,3.798852,3.303161e-08,76.351034
407,0.8290,240.9402,NaN,0.245,4443.24113,622.94368,1.31,13.4054,11.739158,247.659,...,4443.24113,0.000915,0.082627,30.002354,10.519664,2.783006e-08,77.459715,10.157579,3.588528e-08,67.990914
408,2.5170,1282.4100,NaN,0.294,NaN,1303.09647,1.29,13.4054,11.976042,73.800,...,1303.09647,0.000009,0.867000,30.082300,31.179488,1.556190e-09,104.234882,31.235717,1.703209e-09,101.666302


In [3]:
def MakeInteractiveSeparationContrastPlotOfNearbyRVPlanets(orbits, plotx, ploty, phases, 
                                                           saveplot = True, 
                                                           sepau = None,
                                                           sepmas = None,
                                                           filt = 'None', xaxis_label = '',
                                                           annotation_text = '', IWA = 2,
                                                           ytop = 2e-6, ybottom = 2e-10,
                                                           xright = 20, xleft = 0,
                                                           ncolors = 10, ticklocs = 'None', ticklabels = 'None',
                                                          output_file_name = 'RVPlanetContrastPlot'):


    rad = orbits['Re'].copy()
    spt = orbits['SpT Number'].copy()
    
    plotx, ploty = np.array(plotx),np.array(ploty)
    multiplier = 2
    datadf = pd.DataFrame(data={'plotx':plotx, 'ploty':ploty, 'color':spt, 'markersize':rad*multiplier,
                               'name':orbits['pl_name'], 'rad':rad, 'spt':spt, 'dist':orbits['sy_dist'],
                                'phases':phases, 'plotx_og':plotx, 'ploty_og':ploty, 'iwa': 2, 
                                'sepau':sepau, 'sepmas':sepmas
                               })
    datadict = datadf.to_dict(orient = 'list')

    from bokeh.plotting import figure, show, output_file, save
    from bokeh.io import output_notebook
    from bokeh.models import LinearColorMapper, ColumnDataSource, LinearInterpolator
    from bokeh.models import  Range1d, LabelSet, Label, ColorBar, FixedTicker, Span
    from bokeh.models import CustomJS, Slider
    from bokeh.layouts import column, row
    from bokeh.palettes import Magma256, Turbo256, brewer
    from bokeh.transform import linear_cmap
    #output_notebook()


    data=ColumnDataSource(data=datadict)


    tools = "hover, zoom_in, zoom_out, save, undo, redo, reset, pan"
    tooltips = [
        ('Planet', '@name'),
        #("(x,y)", "($x, $y)"),
        ('Cont', '@ploty'),
        ('Phase [deg]', '@phases{0}'),
        ('Sep [au]', '@sepau{0.00}'),
        ('Sep [mas]', '@sepmas{0.00}'),
        ('Rad [Rearth]','@rad{0.00}'),
        ('SpT','@spt{0.0}'),
        ('Dist [pc]','@dist{0.0}')
    ]

    p = figure(width=900, height=750, y_axis_type="log", tools=tools, 
               tooltips=tooltips, toolbar_location="above")

#     mapper = linear_cmap(field_name='color', 
#                          #palette=Magma256,
#                          palette=Turbo256[::-1],
#                          low=min(spt), high=max(spt),
#                         low_color=Turbo256[::-1][150], high_color=Turbo256[::-1][200])
    
    mapper = linear_cmap(field_name='phases', 
                         palette=brewer['PuOr'][ncolors],
                         #palette=Magma256,
                         #palette=Turbo256[::-1],
                         #low=min(phases), high=max(phases))
                         low=20, high=150)
    
    p.circle('plotx','ploty', source=data, fill_alpha=0.8, size='markersize', 
             line_color=mapper, color=mapper)

    
    color_bar = ColorBar(color_mapper=mapper['transform'], width=15, 
                         location=(0,0), title="Phase",
                        title_text_font_size = '20pt',
                         major_label_text_font_size = '15pt')
    
    if ticklocs == 'None':
        ticklocs = np.arange(0,180,20)
        ticklabels = {}
        [ticklabels.update({ticklocs[i]:str(np.round(ticklocs[i],decimals=-1)).replace('.0','')}) for i in range(len(ticklocs))]
    color_bar.ticker=FixedTicker(ticks=ticklocs)
    color_bar.major_label_overrides = ticklabels

    p.add_layout(color_bar, 'right')

    label = Label(
        text= annotation_text,
        x=50, y=20,
        x_units="screen", y_units="screen",text_font_size = '20pt'
    )
    p.add_layout(label)
    
    delt = np.log10(ytop) - np.log10(ybottom)
    
    x,y = 16, 10**(np.log10(ybottom) + (0.9*delt))
    p.circle(x,y, fill_alpha=0.6, size=11*multiplier,
             color='black')
    label1 = Label(x=x, y=y, text=r'\[ 11 R_\oplus\]',
                       x_offset=20, y_offset=-20,
                       text_font_size = '20pt')
    p.add_layout(label1)
    
    x,y = 16, 10**(np.log10(ybottom) + (0.85*delt))
    p.circle(x,y, fill_alpha=0.6, size=5*multiplier, 
             color='black')
    label2 = Label(x=x, y=y, text=r'$$5 R_\oplus$$',
                      x_offset=20, y_offset=-20,text_font_size = '20pt')
    p.add_layout(label2)
    x,y = 16, 10**(np.log10(ybottom) + (0.8*delt))
    p.circle(x,y, fill_alpha=0.6, size=1*multiplier, 
             color='black')
    label3 = Label(x=x, y=y, text=r'$$1 R_\oplus$$',
                      x_offset=20, y_offset=-20,text_font_size = '20pt')
    p.add_layout(label3)

    p.xaxis.axis_label = xaxis_label
    p.yaxis.axis_label = r'\[ \mathrm{Planet/Star\; Reflected\; Light\; Flux\; Ratio} \]'
    p.xaxis.axis_label_text_font_size = '20pt'
    p.yaxis.axis_label_text_font_size = '20pt'
    p.yaxis.major_label_text_font_size = "15pt"
    p.xaxis.major_label_text_font_size = "15pt"
    
    iwa = Span(location=IWA,
                              dimension='height', line_color='grey',
                              line_dash='dashed', line_width=3)
#     iwa = Span(location=data['iwa'],
#                                dimension='height', line_color='grey',
#                                line_dash='dashed', line_width=3)
    p.add_layout(iwa)
    
    p.x_range=Range1d(xleft,xright)
    p.y_range=Range1d(ybottom,ytop)
    

    AgSlider = Slider(start=0.05, end=1.0, value=0.3, step=.01, title="Geometric Albedo")
    IWASlider = Slider(start=1, end=10, value=2, step=.5, title="IWA")
    LambdaSlider = Slider(start=400, end=2000, value=800, step=50, title="Wavelength [nm]")
    DSlider = Slider(start=2, end=34, value=25.4, step=0.5, title="Primary Mirror Diameter [m]")
#         
#        var newx = x.map(d => d * 6.3/newlod );
    sliders_callback_code = """
        var Ag = Ag.value;
        var Lambda = Lambda.value;
        var D = D.value;
        
        var lod = 6.3;
        var newlod = ((Lambda/1000) / D) * 1000
        
        var y = source.data['ploty_og'];
        var x = source.data['plotx_og'];
        var newy = y.map(m => m * Ag/0.3 );
        var newx = x.map(b => b * 800/Lambda );
        var newx = newx.map(d => d * D/25.4 );


        console.log(newy)
        console.log(newx)
        source.data['ploty'] = newy;
        source.data['plotx'] = newx;
        source.change.emit();
    """

    slider_args = dict(source=data, Ag=AgSlider, Lambda=LambdaSlider, D=DSlider)
    
    AgSlider.js_on_change('value', CustomJS(args=slider_args,code=sliders_callback_code))
    LambdaSlider.js_on_change('value', CustomJS(args=slider_args,code=sliders_callback_code))
    DSlider.js_on_change('value', CustomJS(args=slider_args,code=sliders_callback_code))


    #show(row(p, column(AgSlider)))
    #show(column(p, row(AgSlider),row(LambdaSlider),row(DSlider)))
    output_file(output_file_name+".html")
    save(column(p, row(AgSlider),row(LambdaSlider),row(DSlider)))

    
#     if saveplot:
#         output_file(output_file_name+".html")
#         save(p)
#     else:
#         show(p)
#         pass

    return p

orbits = pd.read_csv('GMagAO-X-RV-targets-with-orbital-params.csv')
MakeInteractiveSeparationContrastPlotOfNearbyRVPlanets(orbits, 
                       np.array(orbits['separation_at_average_separation']),
                       np.array(orbits['contrast_at_average_separation']), 
                       np.array(orbits['phase_at_average_separation']),
                       sepau = np.array(orbits['avg sep in au']),
                       sepmas = np.array(orbits['avg sep in mas']),
                       filt = "i'",
                       xaxis_label = r'\[ \mathrm{Avg\; Projected\; Separation}\; [\lambda/D]\]',
                       #annotation_text = r"$$\mathrm{For}\; i'\; filter, \;D = 25.4m\$$",
                       annotation_text = '',
                       saveplot = False,
                       output_file_name = 'RVPlanets-Average-separation-contrast')

figure(id='p1004', ...)

In [57]:
len(orbits)

410

In [4]:
orbits.columns

Index(['pl_orbsmax', 'pl_orbper', 'pl_orbincl', 'pl_orbeccen', 'pl_masse',
       'pl_msinie', 'st_mass', 'sy_dist', 'Re', 'pl_orblper', 'pl_name',
       'SpT Number', 'pl_mass_or_msini_e', 'contrast_at_highest_contrast',
       'separation_at_highest_contrast', 'phase_at_highest_contrast',
       'separation_at_largest_separation', 'contrast_at_largest_separation',
       'phase_at_largest_separation', 'separation_at_average_separation',
       'contrast_at_average_separation', 'phase_at_average_separation',
       'avg sep in au', 'avg sep in mas', 'highest contrast sep in au',
       'highest contrast sep in mas', 'highest sep in au',
       'highest sep in mas'],
      dtype='object')

In [5]:
ticks = np.arange(0,150,20)
ticklabels = {}
[ticklabels.update({ticks[i]:str(np.round(ticks[i],decimals=-1)).replace('.0','')}) for i in range(len(ticks))]
orbits = pd.read_csv('GMagAO-X-RV-targets-with-orbital-params.csv')
MakeInteractiveSeparationContrastPlotOfNearbyRVPlanets(orbits, 
                                                       orbits['separation_at_highest_contrast'],
                                                       orbits['contrast_at_highest_contrast'], 
                                                       orbits['phase_at_highest_contrast'],
                                                       sepau = np.array(orbits['highest contrast sep in au']),
                                                       sepmas = np.array(orbits['highest contrast sep in mas']),
                                                       filt = "i'",
                                                       xaxis_label = r'\[ \mathrm{Avg\; Projected\; Separation}\; [\lambda/D]\]',
                                                       #annotation_text = r"$$\mathrm{For}\; i'\; filter, \;D = 25.4m\$$",
                                                       annotation_text = '',
                                                       saveplot = False,
                                                       ncolors=8, ticklocs = ticks, ticklabels = ticklabels,
                                                       ytop = 2e-2, ybottom = 5e-8,
                                                       xright = 4, xleft = 0,
                                                      output_file_name = 'RVPlanets-separation-highest-contrast')

/var/folders/tm/1xdh6wyn1qv7mhy_c10d5c_w0000gn/T/ipykernel_48834/694087296.py:77: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if ticklocs == 'None':


figure(id='p1264', ...)

In [6]:
orbits = pd.read_csv('GMagAO-X-RV-targets-with-orbital-params.csv')
MakeInteractiveSeparationContrastPlotOfNearbyRVPlanets(orbits, 
                                                       orbits['separation_at_largest_separation'],
                                                       orbits['contrast_at_largest_separation'], 
                                                       orbits['phase_at_largest_separation'],
                                                       sepau = np.array(orbits['highest sep in au']),
                                                       sepmas = np.array(orbits['highest sep in mas']),
                                                       filt = "i'",
                                                       xaxis_label = r'\[ \mathrm{Avg\; Projected\; Separation}\; [\lambda/D]\]',
                                                       #annotation_text = r"$$\mathrm{For}\; i'\; filter, \;D = 25.4m\$$",
                                                       annotation_text = '',
                                                       saveplot = False,
                                                      output_file_name = 'RVPlanets-Max-separation-contrast')

figure(id='p1524', ...)

In [68]:
len(np.where(np.isnan(orbits['pl_orbincl']))[0])

389